# Reprojecting to EPSG 3338

The purpose of this notebook is reproject output GeoTIFFs to EPSG:3338 - and the reason this is split out into a separate notebook is because different resampling techniques could have a significant impact on how variables like air temperature or snow melt for example, would represent variation in elevation where intra-grid and inter-grid cell elevation variance is high. Reprojection choices could also impact data availability for coastal regions.

There are two axes of variation to consider here: resampling, and grid alignment.

## Resampling
Reprojection always involves resampling because the grid is changing. Typically we use the nearest neighbor method - this is computationally cheap and nearest neighbor is the typical default for most libraries and programs that we use. For an example, in nearest neighbor resampling the temperature value assigned to each output cell will be the same as the temperature value at the center of the corresponding (nearest) input cell. Nearest neighbor is a conservative choice because there will be no values in the resampled dataset that do not exist in the source dataset - which is important if you are dealing with measurementsor wish to retain total fidelity to the values in the source dataset. But most of our data are not measurements, so are we tied to nearest neighbor?

Bilinear or cubic resampling would compute some kind of weighted value based on the distance between each input cell and the center of the output cell. This means that the output raster might better capture the temperature variation associated with changes in elevation within and around each input grid cell. However that kind of resampling can introduce smoothing and interpolation artifacts.

## Grid Alignment
We can align output data to a "standard grid" - meaning that the extents will be some integer multiple of the specified output resolution.

## Implementations
There are several ways we implement these different methods as well: GDAL, either from the command line, in a shell script, or spawned from Python. We can also use rasterio and dask!

Let's take a look at the `gdalinfo` output for the data that we have created so far.

In [1]:
from config import OUTPUT_DIR, aux_dir
import subprocess

In [2]:
# just grab a single test raster
input_raster = OUTPUT_DIR / "runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif"

In [3]:
# call gdalinfo and capture the output
gdalinfo_output = subprocess.check_output(['gdalinfo', input_raster])
print(gdalinfo_output.decode())

Driver: GTiff/GeoTIFF
Files: /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif
Size is 299, 209
Coordinate System is:
PROJCRS["unknown",
    BASEGEOGCRS["unknown",
        DATUM["unknown",
            ELLIPSOID["unknown",6370000,0,
                LENGTHUNIT["metre",1,
                    ID["EPSG",9001]]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433,
                ID["EPSG",9122]]]],
    CONVERSION["Polar Stereographic (variant B)",
        METHOD["Polar Stereographic (variant B)",
            ID["EPSG",9829]],
        PARAMETER["Latitude of standard parallel",64,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8832]],
        PARAMETER["Longitude of origin",-150,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8833]],
        PARAMETER["False easting",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]]

The most basic reprojection would look like this:

```sh
gdalwarp -t_srs EPSG:3338 -multi -wo NUM_THREADS=32 input_raster output_raster
```

We shouldn't need to specify nodata values here - those will be read and assigned automatically from the source dataset. 
Let's spawn this command as a subprocess from the notebook here, just like we did with `gdalinfo`.

In [4]:
output_raster = aux_dir / f"epsg3338_{input_raster.name}"

command = f"gdalwarp -t_srs EPSG:3338 -multi -wo NUM_THREADS=32 -overwrite {input_raster} {output_raster}"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

stdout, stderr = process.communicate()
print(stdout.decode())
print(stderr.decode())
gdalinfo_output = subprocess.check_output(['gdalinfo', output_raster])
print(gdalinfo_output.decode())

Creating output file that is 318P x 224L.
Processing /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
Copying nodata values from source /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif to destination /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


Driver: GTiff/GeoTIFF
Files: /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif
Size is 318, 224
Coordinate System is:
PROJCR

OK so the thing to notice here is that the corner coordinates and origin and pixel sizes all are messy floats. And we've gone from an array that 299 X 209 to an array that is 318 X 224. What happens if we now specify that the target resolution should be exactly 12000 m X 12000 m?

In [5]:
output_raster = aux_dir / f"epsg3338_12km_{input_raster.name}"

command = f"gdalwarp -t_srs EPSG:3338 -tr 12000 12000 -multi -wo NUM_THREADS=32 -overwrite {input_raster} {output_raster}"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

stdout, stderr = process.communicate()
print(stdout.decode())
print(stderr.decode())
gdalinfo_output = subprocess.check_output(['gdalinfo', output_raster])
print(gdalinfo_output.decode())

Creating output file that is 316P x 223L.
Processing /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
Copying nodata values from source /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif to destination /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_12km_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


Driver: GTiff/GeoTIFF
Files: /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_12km_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif
Size is 316, 223
Coordinate System 

As expected, the pixel size is now truly 12 km in each dimension, but now the array size is 316 X 223! Next let's experiment with aligning the 12 km pixels to the "standard" grid, meaning that the coordinates should all have integer values that are multiples of the pixel size. This is done with the GDAL `-tap` option (which must be used in conjuction with `-tr`).

In [6]:
%%time
output_raster = aux_dir / f"epsg3338_12km_tap_nearest_{input_raster.name}"

command = f"gdalwarp -t_srs EPSG:3338 -tap -tr 12000 12000 -multi -wo NUM_THREADS=32 -overwrite {input_raster} {output_raster}"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

stdout, stderr = process.communicate()
print(stdout.decode())
print(stderr.decode())
gdalinfo_output = subprocess.check_output(['gdalinfo', output_raster])
print(gdalinfo_output.decode())

Creating output file that is 317P x 224L.
Processing /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
Copying nodata values from source /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif to destination /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_12km_tap_nearest_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


Driver: GTiff/GeoTIFF
Files: /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_12km_tap_nearest_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif
Size is 317

Now that is a rather clean `gdalinfo` output - and the array size is 317, 224. It does seem that these options `-multi -wo NUM_THREADS=32` speed up the processing slightly. This is the exact equilvalent of executing `gdalwarp` from the command line like this:

```sh
gdalwarp -t_srs EPSG:3338 -tap -tr 12000 12000 -multi -wo NUM_THREADS=32 input_raster output_raster
```
or the one-line incantation to do this for all GeoTIFFs in a folder:

```sh
find source_dir -name "*.tif" -exec gdalwarp -t_srs EPSG:3338 -tap -tr 12000 12000 -multi -wo NUM_THREADS=32 {} output_dir/{} \;
```

The above implemenation won't tweak the filenames, but will just write the data using the input filename to a different directory.


#### Resampling

Now, all of these implementations use the same resampling method: nearest neighbor. GDAL provides quite a few alternatives - most relevant for our work are likely the following options:

`bilinear`: bilinear resampling.

`cubic`: cubic resampling.

`cubicspline`: cubic spline resampling.

Let's run a few of these with our most recent version of the gdalwarp command:

In [7]:
resampling_methods = ["bilinear", "cubic", "cubicspline"]

for method in resampling_methods:
    
    output_raster = aux_dir / f"epsg3338_12km_tap_{method}_{input_raster.name}"
    
    command = f"gdalwarp -t_srs EPSG:3338 -tap -tr 12000 12000 -multi -wo NUM_THREADS=32 -overwrite -r {method} {input_raster} {output_raster}"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    stdout, stderr = process.communicate()
    print(stdout.decode())
    print(stderr.decode())

Creating output file that is 317P x 224L.
Processing /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
Copying nodata values from source /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif to destination /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/auxiliary_content/epsg3338_12km_tap_bilinear_runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


Creating output file that is 317P x 224L.
Processing /atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries/runoff_mm_CSIRO-Mk3-6-0_rcp85_aug_total_2050-2060_mean.tif [1/1] : 0Using internal nodata

In [8]:
import rasterio as rio
from rasterio.windows import Window
from pyproj import Transformer
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/ua-snap/geospatial-vector-veracity/main/vector_data/point/alaska_point_locations.csv").drop(["region", "country"], axis=1)

In [10]:
df.set_index("id", inplace=True)

In [11]:
# function to extract values from a geotiff
def extract_points(fp, lat_lons):
    with rio.open(fp) as src:
        # get CRS
        tiff_crs = src.crs

        # reproject point lat lon
        transformer = Transformer.from_crs("epsg:4326", tiff_crs)
        
        extracted_values = []
        for loc in lat_lons:
            lat, lon = loc
            x, y = transformer.transform(lat, lon)

            # get row / column
            r, c = src.index(x, y)

            # create a window object by specifying the starting place 
            #  of the array and the size of the window
            # note order is column offset, row offset, width, height
            window = Window(c, r, 1, 1)

            # read the data
            arr = src.read(1, window=window)
            extracted_values.append(arr[0])
    return extracted_values, fp.name

In [12]:
paths = list(aux_dir.glob("*.tif"))

for tiff in paths:
    # apply extract_points to each row of the dataframe
    df[tiff.name.split("runoff")[0]] = df.apply(lambda row: extract_points(tiff, [(row['latitude'], row['longitude'])])[0][0], axis=1)

In [13]:
df

,name,alt_name,latitude,longitude,km_distance_to_ocean,epsg3338_,epsg3338_12km_,epsg3338_12km_tap_nearest_,epsg3338_12km_tap_bilinear_,epsg3338_12km_tap_cubic_,epsg3338_12km_tap_cubicspline_
id,,,,,,,,,,,
AK1,Afognak,Agw’aneq,58.0078,-152.7680,0.2,[24.0],[21.0],[26.0],[24.176962],[24.176962],[24.131104]
AK2,Akhiok,Kasukuak,56.9455,-154.1700,0.1,[29.0],[31.0],[31.0],[29.730608],[30.459072],[29.404856]
AK3,Akiachak,Akiacuar,60.9094,-161.4310,89.7,[9.0],[9.0],[8.0],[8.254271],[7.8803988],[8.952019]
AK4,Akiak,Akiaq,60.9122,-161.2140,97.4,[10.0],[10.0],[10.0],[10.03617],[10.039776],[9.969265]
AK5,Akutan,Achan-ingiiga,54.1385,-165.7780,0.6,[-9999.0],[-9999.0],[-9999.0],[-9999.0],[-9999.0],[-9999.0]
...,...,...,...,...,...,...,...,...,...,...,...
AK432,Womens Bay,NaN,57.7099,-152.5860,1.1,[27.0],[41.0],[27.0],[27.322186],[27.322186],[28.241877]
AK433,Woody Island,NaN,57.7800,-152.3550,0.4,[24.0],[-9999.0],[24.0],[24.20971],[24.20971],[24.52939]
AK434,Wrangell,Shtax’héen,56.4708,-132.3770,0.6,[-9999.0],[-9999.0],[-9999.0],[-9999.0],[-9999.0],[-9999.0]


In [16]:
raster_cols = ['epsg3338_', 'epsg3338_12km_', 'epsg3338_12km_tap_nearest_', 'epsg3338_12km_tap_bilinear_', 'epsg3338_12km_tap_cubic_', 'epsg3338_12km_tap_cubicspline_']

In [ ]:
df_filtered = df[df.apply(lambda x: np.any(np.array(x) == [-9999.0]), axis=1)]
df_filtered

In [ ]:
df_filtered.drop(["region", "country"], axis=1)

In [ ]:
paths = list(aux_dir.glob("*.tif"))

for projected in paths:
    src = rio.open(projected)
    arr = src.read(1)
    arr[arr==src.nodata]=np.nan
    plt.figure()
    plt.imshow(arr, vmin=0, vmax=30000)

Compare variance for different resampling techniques.
Export list of communities that have no data in any technique:
Export list of those that have data for some techniques but not others..



### Juneau (12 km resolution) CCSM4 RCP 8.5: 2080-2090 Average May total snow melt (mm)
 - Nearest Neighbor: 0 mm
 - Median: 0 mm
 - Mean: 190 mm
 - Bilinear: 193 mm
 - Cubic: 229 mm
 
There is another axis of variation when reprojecting 


Another point to consider is that summarizing data by HUC-12s, rather than discrete points, is one way to buffer against the impact of the elevation assigned by the source model to a specific grid cell.



In [ ]:
from pathlib import Path
import os

OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR"))
paths = list(OUTPUT_DIR.glob("*.tif"))
assert len(paths) == 14400

In [ ]:
import dask
import dask.distributed as dd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_origin

# Create a Dask client

In [ ]:
client = dd.Client()


In [ ]:


# Define the function to reproject a single raster file
def reproject_raster(file):
    # Open the input raster
    with rasterio.open(file) as src:
        # Define the output CRS and transform
        dst_crs = 'EPSG:3338'
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)

        # Define the output file name
        out_file = OUTPUT_DIR / "cog" / file.name

        # Define the output raster profile
        out_profile = src.profile.copy()
        out_profile.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'compress': 'deflate',
            'predictor': 2,
            'tiled': True,
            'blockxsize': 512,
            'blockysize': 512
        })

        # Create the output raster
        with rasterio.open(out_file, 'w', **out_profile) as dst:
            # Reproject the input raster to the output CRS
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                num_threads=1 # Set to 1 to avoid issues with Dask
            )



In [ ]:

# Use Dask to parallelize the processing of the input raster files
dask.compute(*[dask.delayed(reproject_raster)(f) for f in paths])


In [ ]:
However, unless we hear otherwise from the downscaling group, or perhaps Jeremy L., the the move is to stick with nearest neighbor.
